<table>
    <tr>
        <td style="text-align:left">
            <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9ItLTT_F-3Q30cu7ZCCoKmuFGBt22pe7pNA" alt="Logo Universidad" width="300"/>
        </td>
        <td>
            Departamento de Ciencias de la Computación y de la Decisión<br>
            Facultad de Minas<br>
            Universidad Nacional de Colombia<br>
            Optimizacion e IA 2024-2S<br><br>
            Docente: Maria Constanza Torres Madronero<br>
            <br>
            Contribuciones a la guia por: <br>
            - Deimer Miranda Montoya (2023)<br>
            - Luis Fernando Becerra Monsalve (2024)
        </td>    
        </td>    
    </tr>
</table>

### Clustering
Una tarea frecuente en aprendizaje de máquinas no supervisado es el agrupamiento o clustering. Estas técnicas permiten descubrir relaciones o patrones ocultos en los datos analizados. Existen diferentes métodos de agrupamiento, destacándose Kmeans por su simplicidad. En esta práctica compararemos Kmeans con técnicas de clustering que emplean métodos metaheuristicos como GA, PSO y ACO.

Primero generaremos una nube de datos para comparar los diferentes algoritmos. Para esto usaremos nuevamente herramientas disponibles en la librería sklearn.

In [ ]:
#Importar librerias
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn import preprocessing

In [ ]:
#Generacion de nubes de puntos


#Repetir con random_state=2
#Repetir con random_state=37 y num_cluster=5

#Importante: normalizar los datos

#Visualizacion de los datos
colors = ["#4EACC5", "#FF9C34", "#4E9A06", 'm']
for k, col in enumerate(colors):
    cluster_data = y_true == k
    plt.scatter(X[cluster_data, 0], X[cluster_data, 1], c=col, marker=".", s=10)

# Kmeans
Para esta práctica usaremos la implementación clásica de Kmeans empleando el método disponible en la librería sklearn.

In [ ]:
####

#Coeficiente de silueta
Vamos a usar el [coeficiente de silueta](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) como métrica de desempeño de los algoritmos de clustering. Existen diferentes métricas para esto, como la suma de la distancia al cuadrado de cada punto a su centroide. Sin embargo, el coeficiente de silueta no solo tiene en cuenta la distancia intra-cluster (distancia de cada punto al centro del clúster: a), sino también la distancia al clúster más cercano (b). El coeficiente de silueta para una muestra está dado por (b - a) / max(a, b). El mejor valor que se puede obtener es 1 y el peor -1. Valores cercanos a 0 indican clústeres que se sobreponen. Valores negativos indican una mala asignación de muestras a los clústeres, dado que existe un clúster más cercano o simular a ciertas muestras.

In [ ]:
#Calculo del coeficiente de silueta
###
print("Coeff Silueta para Kmeans: = ", score)

In [ ]:
#Grafica de los cluster y centroides obtenidos
for k, col in enumerate(colors):
    filtered = X[kmeans.labels_==k]
    plt.scatter(filtered[:, 0], filtered[:, 1], c=col, marker=".", s=10)
    plt.scatter(kmeans.cluster_centers_[k, 0], kmeans.cluster_centers_[k, 1], c="black", marker="o", s=20)

### Algoritmo Genético
Vamos a usar la libreria PyGAD para realizar el proceso de clustering.

In [ ]:
#Instalacion de la libreria


In [ ]:
#Importar libreria pygad


In [ ]:
#Paso 1: Definire funcion fitness (funcion de aptitud)
#solution: arreglo con los centroides del cluster
#solution = [c1(x) c1(x) c2(x) c2(y)... cn(x) cn(y)]

def fitness_func(ga_instance, solution, solution_idx):


    return fitness

In [ ]:
#Paso 2: Preparamos los parametros para correr el algoritmo genetico
num_generations = ##
num_parents_mating = ##
fitness_function = fitness_func
sol_per_pop = ##
#Numero de genes en la solucion
num_genes = num_cluster*2
#Rangos iniciales para la poblacion inicial
init_range_low = ##
init_range_high = ##
#Metodo para la seleccion de padres
#"rws": Ruleta
#"tournament": Torneo
parent_selection_type = "rws"
keep_parents = 5
crossover_type = "single_point"
crossover_probability=0.8
mutation_type = "random"
mutation_probability = 0.1

In [ ]:
#Instanciar el Algoritmo
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       crossover_probability = crossover_probability,
                       mutation_type=mutation_type,
                       mutation_probability=mutation_probability)

In [ ]:
#Corremos el algoritmo
ga_instance.run()

In [ ]:
#Extraemos la mejor solucion
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(f"Parameters of the best solution : {solution}")
print(f"Fitness value of the best solution = {solution_fitness}")


In [ ]:
ga_instance.plot_fitness()

In [ ]:
#Obtenemos los cluster asociados a cada muestra
distance = np.zeros([n_samples,num_cluster])
k=0
for i in np.arange(0,(num_cluster)*2,2):
  centroid = np.array([solution[i], solution[i+1]])
  distance[:,k] = np.square(np.sum(np.power(X - centroid, 2), axis=1))
  k=k+1
GA_labels=distance.argmin(axis=1)

In [ ]:
#Mostrar clusters
i=0
for k, col in enumerate(colors):
    filtered = X[GA_labels==k]
    plt.scatter(filtered[:, 0], filtered[:, 1], c=col, marker=".", s=10)
    plt.scatter(solution[i], solution[i+1], c="black", marker="o", s=20)
    i+=2